In [29]:
# 1. Force upgrade the critical libraries
%pip install -U langchain langchain-core langchain-openai langchain-community pydantic

# 2. IMPORTANT: You must restart the kernel after running this!
# In VS Code/Jupyter: Click "Restart" or "Restart Kernel" in the top toolbar.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [30]:
# 1. Install/Update libraries to ensure compatibility (Run this once)
%pip install -U langchain langchain-openai langchain-community pandas requests pydantic

# 2. Imports
import os
import requests
import pandas as pd
from io import StringIO
from pydantic import BaseModel, Field

# --- LANGCHAIN IMPORTS ---
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

from langchain_classic.agents import AgentExecutor, create_tool_calling_agent

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [36]:
import os
import requests
import pandas as pd
from io import StringIO
from langchain_classic.agents import AgentExecutor, create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate

# 1. Connect to Local LLM (Qwen/Mistral)
llm = ChatOpenAI(
    base_url="http://127.0.0.1:1234/v1",
    api_key="lm-studio",
    model="local-model",
    temperature=0,
    streaming=True
)

# 2. Define the Tool (Standard)
@tool
def fetch_bom_dataset(url: str) -> str:
    """
    Downloads a CSV dataset from a URL (e.g., Bureau of Meteorology) and returns a summary.
    Input should be the full URL string.
    """
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        
        # Parse CSV
        content = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(content), on_bad_lines='skip')
        
        return (
            f"SUCCESS: Downloaded data from {url}\n"
            f"Shape: {df.shape}\n"
            f"Columns: {list(df.columns)}\n"
            f"First 5 rows:\n{df.head().to_string()}"
        )
    except Exception as e:
        return f"ERROR: {str(e)}"

tools = [fetch_bom_dataset]

# 3. Define the ReAct Prompt (Hardcoded for stability)
# This teaches the model explicitly how to think and act.
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

# 4. Create the ReAct Agent
# This uses simple text generation, avoiding the Pydantic/Tool Binding error completely.
agent = create_react_agent(llm, tools, prompt)

# 5. Create the Executor
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    handle_parsing_errors=True # IMPORTANT for local models
)

print("✅ ReAct Agent built successfully.")

✅ ReAct Agent built successfully.


In [37]:
test_url = "https://raw.githubusercontent.com/gramener/datasets/refs/heads/main/card_transactions.csv"
query = f"Download the dataset from {test_url} and tell me the columns."

response = agent_executor.invoke({"input": query})
print("\n--- FINAL ANSWER ---")
print(response['output'])



> Entering new AgentExecutor chain...
I need to download the dataset from the provided URL and then examine its columns.

Action: fetch_bom_dataset
Action Input: "https://raw.githubusercontent.com/gramener/datasets/refs/heads/main/card_transactions.csv"I need to download the dataset from the provided URL and then examine its columns.

Action: fetch_bom_dataset
Action Input: "https://raw.githubusercontent.com/gramener/datasets/refs/heads/main/card_transactions.csv"SUCCESS: Downloaded data from https://raw.githubusercontent.com/gramener/datasets/refs/heads/main/card_transactions.csv
Shape: (10000, 21)
Columns: ['ID', 'Transaction Time', 'Transaction Type', 'Channel', 'Is 3DS', 'Is Token', 'Decision', 'Decline Reason Code', 'Issuer Name', 'Issuer Country', 'Issuer Region', 'Funding source', 'Acquirer Name', 'Acquirer Country', 'Acquirer Region', 'Jurisdiction', 'Is Dispute', 'Is Fraud', 'Dispute Type', 'Amount', 'Industry Segment']
First 5 rows:
                ID          Transaction T